# Visualize Effects of Noise on Camera Poses (lego)

This notebook:

- Loads camera transforms for the NeRF Synthetic Blender **lego** scene.
- Applies simple Gaussian noise to camera extrinsics.
- Visualizes original vs noisy camera pose clouds in 3D.

Noise utilities are imported from `scripts/inject_noise.py`.


In [2]:
from pathlib import Path
import json

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import numpy as np

from scripts.inject_noise import perturb_transforms_dict

# Paths
PROJECT_ROOT = Path("..").resolve()
SCENE_DIR = PROJECT_ROOT / "data" / "raw" / "lego"
print("Scene directory:", SCENE_DIR)

# Load transforms
candidates = [
    SCENE_DIR / "transforms.json",
    SCENE_DIR / "transforms_train.json",
]
transforms_path = None
for path in candidates:
    if path.exists():
        transforms_path = path
        break

if transforms_path is None:
    raise FileNotFoundError(
        f"No transforms.json or transforms_train.json found in {SCENE_DIR}"
    )

with transforms_path.open("r") as f:
    transforms = json.load(f)

print("Loaded transforms from:", transforms_path)

# Apply noise to poses
noisy_transforms = perturb_transforms_dict(
    transforms,
    translation_std=1e-3,
    rotation_std_deg=1.0,
)



ModuleNotFoundError: No module named 'scripts'

In [ ]:
# Visualize original vs noisy camera positions

def extract_positions(transforms_dict):
    positions = []
    for frame in transforms_dict.get("frames", []):
        mat = np.array(frame["transform_matrix"], dtype=np.float32)
        positions.append(mat[:3, 3])
    if not positions:
        raise ValueError("No frames found in transforms JSON.")
    return np.stack(positions, axis=0)


orig_positions = extract_positions(transforms)
noisy_positions = extract_positions(noisy_transforms)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.scatter(
    orig_positions[:, 0],
    orig_positions[:, 1],
    orig_positions[:, 2],
    c="b",
    s=10,
    label="original",
)
ax.scatter(
    noisy_positions[:, 0],
    noisy_positions[:, 1],
    noisy_positions[:, 2],
    c="r",
    s=10,
    marker="x",
    label="noisy",
)

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Original vs noisy camera centers")
ax.legend()
plt.tight_layout()
plt.show()

